<h1 style="font-size:40px;"><center>Exercise I:<br> Backpropagation
</center></h1>


In [1]:
%load_ext autoreload
%autoreload 2

# Theory

Tutorial from pytorch: https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html
* What is torch, torchvision?
* What is a tensor?
* How does the neural network work?
- Define optimizer
- Define loss

- Forward pass
- Backward pass
- gradient descent (step)

# Common pitfalls


# Code

In [2]:
# local modules
from dataset import CustomDataLoader 
from config import flying_objects_config

# Torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F

## Load config

In [3]:
cfg = flying_objects_config()

## Setup Dataloader

In [4]:
dataset = CustomDataLoader(**cfg)
trainset = dataset.train_dataloader()

In [5]:
dataset.train_dataloader().dataset.data.shape

(10817, 128, 128, 3)

## Define neural network

In [30]:
class Net(nn.Module):

    def __init__(self, channels=3, num_classes=9):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(channels, 64, 3)
        self.conv2 = nn.Conv2d(64, 128, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(128 * 30 * 30, 256)  # 128*128 from image dimension: Filters*shape_x*shape_y
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        print(x.shape)
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        print(x.shape)
        x = x.view(-1, self.num_flat_features(x))
        print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        print(x.shape)
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        print(num_features)
        return num_features

## Define loss and optimizer

In [31]:
model = Net(channels=cfg.IMAGE_CHANNEL, num_classes=cfg.NUM_CLASS)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

## Show metadata

In [32]:
model

Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=115200, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=12, bias=True)
)

In [33]:
# Learnable parameters?

"""'
# Settings
- Learning rate?
- Classes?
- Number of files?

"""

"'\n# Settings\n- Learning rate?\n- Classes?\n- Number of files?\n\n"

In [34]:
import h5py
df = h5py.File("../data/flyingData_lab1.hdf5", 'r')

data = df['training']['data']
labels = df['training']['label']

## Learning framework

In [35]:
for epoch in range(cfg.NUM_EPOCHS):  
    running_loss = 0.0
    for i, data in enumerate(trainset, 0):
        # Get input and target. Contains a minibatch!
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward 
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        
        # backward propagate
        loss.backward()
        
        # gradient descent
        optimizer.step()

        # Calculate loss
        running_loss += loss.item()
        
        #
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

torch.Size([32, 3, 128, 128])
torch.Size([32, 128, 30, 30])
torch.Size([32, 128, 30, 30])
115200
torch.Size([32, 115200])


RuntimeError: 1D target tensor expected, multi-target not supported

# The report!


### Name

### Introduction

### Answers to questions

### Summary
  